In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_gpy_sparse import *
from parameters import *
import GPy

import matplotlib.pyplot as plt
np.random.seed(101)


In [2]:
parameters['i_end'] = 100
parameters


{'i_start': 0,
 'i_end': 100,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.35it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Failed to load : ../data/temp_data/cache_0_100_None/loc_Tracer.pkl


UnboundLocalError: local variable 'data_df' referenced before assignment

In [ ]:
# Data for the regression
dim = 3
t = 100
X = loc_df.values #[I,:dim]
Y = data_df.values[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Y.shape

In [ ]:
m_inducing = 100
kernel = GPy.kern.Matern52(dim,ARD=False) + GPy.kern.White(dim)
model_gp = GPy.models.SparseGPRegression(X,Y,kernel,num_inducing=m_inducing)

In [ ]:
model_gp.plot_inducing(visible_dims=[0,2])

In [ ]:
model_gp.optimize(messages=True,max_iters = 100000)


In [ ]:
model_gp.plot_inducing(visible_dims=[0,1])

In [ ]:
model_gp.plot_data(visible_dims=[0,1],figsize=(10,8))

In [ ]:
model_gp.plot_mean( visible_dims = [0,1], figsize = (10,8), levels = 50 )

## Sensor Optimisation

Define the Sets for the optimisation

In [ ]:
n_V = X.shape[0]
sets = define_sets(n_V)

Number of sensors to place : 

In [ ]:
k = 10

### Lazy Sensor Optimisation

In [ ]:
A_opt = {}

In [ ]:
A_opt['lazy'] = sensor_loc_optimisation_lazy(k, sets, X, Y, model_gp)

In [ ]:
A_opt['lazy'] 

### Naive Sensor Optimisation

In [ ]:
A_opt['naive'] = sensor_loc_optimisation_naive(k, sets, X, Y, model_gp)

In [ ]:
A_opt['naive']

In [ ]:
A = np.zeros((n_V,1))
A[A_opt['naive'],:] = 1

In [ ]:
save_vtu(ref_vtu[0],"A_lazy_k10.vtu",A)

In [5]:
A_ = np.setdiff1d(np.array([1,2,3,4]),np.array([1,2,3,4]) )
A_

array([], dtype=int64)

True